اول حاجه هنحط الداتا في مونجو دي بي 

In [ ]:
#this is new DAG
import json
from pymongo import MongoClient

#  الاتصال بـ MongoDB
client = MongoClient("mongodb://localhost:27017/")

#  اختيار Database و Collection
db = client["nyc_data"]
collection = db["nyc_traffic"]

# (اختياري) تفريغ الكوليكشن لو بتجرب
# collection.delete_many({})

#  قراءة ملف الـ JSON
with open("nyc_traffic.json", "r") as f:
    data = json.load(f)

# تأكيد إن الداتا Array
assert isinstance(data, list), "JSON file must contain a list of documents"

#  إدخال الدhتا
collection.insert_many(data)

print(f" Inserted {len(data)} documents into MongoDB successfully")


✅ Inserted 41362 documents into MongoDB successfully


حولنا الداتا بتتاعت الشوارع من ملف سي اي في اي  ملف  parquet
ودي عشان ننوع من مصارد الداتا الي جايلنا 

In [1]:

import pandas as pd 
data = pd.read_csv("Centerline.csv")
data.to_parquet("streets_data.parquet")

انشاء اول تاسك في الداج وده هنعمل كونكت مع الاس 3 و هنكريت باكت 
ونعمل فيها 2 فولدر وبعدين نرفع الرو داتا فيها 
1- هنجيب الداتا  من مونجو دي بي ونرفعها علي هئيه جيسون فايل 
2- هنجيب الفايل البركيه ونرفعه 
3- هنجيب الفايل الثالث والاخير ونرفعه الي هو csv 

In [12]:
from pymongo import MongoClient
import json
import boto3

# اتصال Mongo
client = MongoClient("mongodb://localhost:27017")
db = client["nyc_data"]
collection = db["nyc_traffic"]

#-----------------------------------------------
# connect with s3 
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9010',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123'
)
Bucket="datalake"

#------------------------------------------------
# create bucket لو مش موجود 
try:
     s3.create_bucket(Bucket="datalake")
except:
     pass

#------------------------------------------------
# create 2 folder (raw , processed)
try:
     s3.put_object(Bucket="datalake", Key="raw/")
     s3.put_object(Bucket="datalake", Key="processed/")
except:
     pass

#----------------------------------------------------------
# upload first file to s3 (json file)
cursor = collection.find({}, {"_id": 0})

data = list(cursor)          # in-memory
json_bytes = json.dumps(
    data,
    ensure_ascii=False
).encode("utf-8")

#load to s3
s3.Bucket("datalake").put_object(
    Key="raw/mongo_traffic.json",
    Body=json_bytes,
    ContentType="application/json"
)


#---------------------------------------------------==
# upload second file csv to s3
s3_client = boto3.client(
    's3',
    endpoint_url='http://localhost:9010',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin123'
)


s3_client.upload_file(
    Filename="Motor_Vehicle_Collisions_Crashes.csv",  # المسار المحلي للفايل
    Bucket="datalake",                    # اسم الباكيت في S3/MinIO
    Key="raw/Motor_Vehicle_Collisions_Crashes.csv",    # المسار داخل الباكيت
    ExtraArgs={"ContentType": "text/csv"}   # دي زي ميتا داتا بعرفه ايه الكونتنت الي جوا الفايل ده 

)

#--------------------------------------
# upload the last file (parquet file )

s3_client.upload_file(
    Filename="streets_data.parquet",  # المسار المحلي للفايل
    Bucket="datalake",                    # اسم الباكيت في S3/MinIO
    Key="raw/streets_data.parquet" ,   # المسار داخل الباكيت
    ExtraArgs={"ContentType": "application/octet-stream"}  # دي زي ميتا داتا بعرفه ايه الكونتنت الي جوا الفايل ده 

)




التاسك التاني هنجيب الداتا بتاعت الجيسون فايل ونحولها لداتا فريم 
دا بستخدام باي سبارك و نعمل عليها بروسيسنج وبعدين نرفعها علي س3 تاني بس المره ده في فولدر اسمه بروسيس داتا 


In [ ]:
# task 2
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , lower ,to_date
spark = (
    SparkSession.builder
    .appName("transform1")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)

#spark.sparkContext.setLogLevel("WARN")
# هنجيب الداتا بقا ونحولها لداتا فريم

df = spark.read.json(
    "s3a://datalake/raw/mongo_traffic.json"
)

#--------------------------------------------------------
# تعديل القيم في الاعمده لتصبح كلها سمول 

cols_to_lower = ["borough", "incident_type", "severity", "status", "street_name", "weather_condition"]

for c in cols_to_lower:
    df = df.withColumn(c, lower(col(c)))

#--------------------------------------------------------
# تغير الداتا تايب بتاع الديت 
df1 = df.withColumn(
    "created_date", to_date(col("created_date"), "yyyy-MM-dd"))

# تغير اسماء الاعمده 
rename_coulmn={
    "fatalities":"pepole_killed",
    "incident_type":"complaint_type",
    "incident_id":"complaint_id",
    "injuries_reported":"pepole_injuries",
}

for  old,new in rename_coulmn.items():
    df1=df1.withColumnRenamed(old,new)

#------------------------------------
df1.write \
    .mode("overwrite") \
    .option("header","true") \
    .csv("s3a://datalake/processed/complaint_311")

df1.show(10)



26/01/11 18:56:57 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/11 18:56:57 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/11 18:56:57 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


+---------+------------+-------------+--------------------+-------------------+---------------+---------+----------+---------------------+--------+------+---------------+--------------------+-----------------+-----------------+
|  borough|created_date|pepole_killed|        complaint_id|     complaint_type|pepole_injuries| latitude| longitude|response_time_minutes|severity|status|    street_name|traffic_volume_level|vehicles_involved|weather_condition|
+---------+------------+-------------+--------------------+-------------------+---------------+---------+----------+---------------------+--------+------+---------------+--------------------+-----------------+-----------------+
|   queens|  2025-01-01|            0|99a27f0e-637a-45d...| signal malfunction|              0|40.758144|-73.800187|                   17|  medium|  open|     2nd avenue|               Light|                3|            clear|
| brooklyn|  2025-01-01|            0|8424a97b-e059-4e6...|    illegal parking|         

26/01/11 22:23:56 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 253631 ms exceeds timeout 120000 ms
26/01/11 22:23:56 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:131)
	at org.apache.spark.storage.BlockManager

In [17]:
df1.printSchema()


root
 |-- borough: string (nullable = true)
 |-- created_date: date (nullable = true)
 |-- fatalities: long (nullable = true)
 |-- incident_id: string (nullable = true)
 |-- incident_type: string (nullable = true)
 |-- injuries_reported: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- response_time_minutes: long (nullable = true)
 |-- severity: string (nullable = true)
 |-- status: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- traffic_volume_level: string (nullable = true)
 |-- vehicles_involved: long (nullable = true)
 |-- weather_condition: string (nullable = true)



التاسك التاني 
ودي عباره عن اننا هنجيب الفايل الي هو csv 
وهنبدء نتعامل معاه علي pyspark 
 بحيث اننا نعمل بروسيسنج لداتا الي فيه بما يتناسب مع احتياجتنا 
 وفي الاخر نرفع الفايل علي هئيه بركيه فايل علي s3
جوا الفولدر الي اسمه processed

In [8]:
# task 3
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import expr
from pyspark.sql.functions import to_date, col ,date_format , to_timestamp , lower



spark = (
    SparkSession.builder
    .appName("transform2")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)
#==============================================================
# هنجيب الداتا  ونحولها لداتا فريم

df = spark.read.csv(
    "s3a://datalake/raw/Motor_Vehicle_Collisions_Crashes.csv",
    inferSchema=True,
    header=True
)



#----------------------------------------------------------
# معالجه الداتا الي في الفايل 
#----------------------------------------------------------



# تحدبد الاعمده الي محتجنيها بس 
df_selected=df.select("COLLISION_ID","CRASH DATE","CRASH TIME","BOROUGH","VEHICLE TYPE CODE 1"
              ,"ON STREET NAME","NUMBER OF PERSONS INJURED","NUMBER OF PERSONS KILLED"
              , "CONTRIBUTING FACTOR VEHICLE 1") 

#----------------------------------
# تغير الداتا تيب بتاع عمود التاريخ والقيم الي مش هتفع تبقا تاريخ تيبقا نل
df1 = df_selected.withColumn(
    "CRASH DATE",
    expr("try_to_date(`CRASH DATE`, 'M/d/yyyy')")
)

#----------------------------------
#  في عمود الديت هنعمل فلتره بحيث نجيب  القيم الي مش بنل 
df2 = df1.filter(col("CRASH DATE").isNotNull())

#----------------------------------
# فلتره الداتا لعام 2025 بس 
df_2025_plus = df2.filter(
    col("CRASH DATE") > "2025-01-01"
)


#----------------------------------
# هنحاول اننا نعدل القيم الفاضيه بنل دي بعدين ان كان في وقت 
# بس دلوقتي هنحط  القيم الفاضيه دي unknown
df_fill = df_2025_plus.fillna({
    "BOROUGH": "UNKNOWN",          # أي Null في BOROUGH هيتحول لـ "UNKNOWN"
    "ON STREET NAME": "UNKNOWN STREET" , # أي Null في ON STREET NAME هيتحول لـ "UNKNOWN STREET"
    "NUMBER OF PERSONS INJURED":"0", # عدد الاصابات الي بنل هحط مكانه 0
    "VEHICLE TYPE CODE 1":"Sedan" # عندي 1417 قيمه  بنل هعوضهم  ب اكتر قيمه اتكرت 
    
})

#----------------------------------------------------------
# تحويل الداتا تايب بتاع العمود الاصابات الي  int 
df_4 = df_fill.withColumn(
    "NUMBER OF PERSONS INJURED",
    col("NUMBER OF PERSONS INJURED").cast("int")
)
from pyspark.sql.functions import to_timestamp , date_format
#----------------------------------
# تحويل الداتا الي في العمود بتاع الوقت لصيغه ديت
df_time = df_4.withColumn(
    "CRASH TIME",
    to_timestamp(col("CRASH TIME"), "H:mm")
)

#----------------------------------
# HH:MM تحديد ان الديت يبقا بصيغه الساعه والدقائق فقط  
df_final = df_time.withColumn(
    "CRASH TIME",
    date_format("CRASH TIME", "HH:MM")
)

#=========================================
# تغير اسماء الاعمده
rename_dict = {
    "CRASH DATE": "crash_date",
    "CRASH TIME": "crash_time",
    "ON STREET NAME": "street_name",
    "NUMBER OF PERSONS INJURED": "persons_injured",
    "NUMBER OF PERSONS KILLED": "persons_killed",
    "VEHICLE TYPE CODE 1": "vehicle_type",
    "CONTRIBUTING FACTOR VEHICLE 1": "contributing_factor",
    "COLLISION_ID": "collision_id",
    "BOROUGH": "borough"
}

for old, new in rename_dict.items():
    df_final = df_final.withColumnRenamed(old, new)

# هنحول الداتا الي في الاعمده كلها لسمول  
df_final = df_final.withColumn("borough", lower(col("borough")))
df_final = df_final.withColumn("vehicle_type", lower(col("vehicle_type")))
df_final = df_final.withColumn("street_name", lower(col("street_name")))
df_final = df_final.withColumn("contributing_factor", lower(col("contributing_factor")))

df_final.show(10)



# هنحفظ الفايل الجديد علي س3 في الفولدر بتاع الداتا المتعالجه 
df_final.write \
    .mode("overwrite") \
    .option("header","true") \
    .csv("s3a://datalake/processed/Motor_Vehicle_Collisions_Crashes")




+------------+----------+----------+---------+------------+-----------------+---------------+--------------+--------------------+
|collision_id|crash_date|crash_time|  borough|vehicle_type|      street_name|persons_injured|persons_killed| contributing_factor|
+------------+----------+----------+---------+------------+-----------------+---------------+--------------+--------------------+
|     4803256|2025-04-02|     09:01|   queens|   ambulance|   unknown street|              0|             0|         unspecified|
|     4804525|2025-04-07|     12:01|    bronx|       sedan|bedford park blvd|              0|             0|         unspecified|
|     4804466|2025-04-07|     06:01|    bronx|       sedan|   unknown street|              0|             0|        unsafe speed|
|     4790397|2025-01-31|     08:01|   queens|       sedan|        exeter st|              1|             0|driver inattentio...|
|     4790246|2025-02-02|     20:01|  unknown|       sedan|   unknown street|             

26/01/10 17:08:29 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:08:29 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:08:29 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:08:39 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:08:39 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:08:48 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


task4
------
هنجيب الداتا للفايل الثالث من s3
وهنعمل بروسيسنج علي الفايل ده برضو 
وبعدين نرفع الفايل تاني علي s3
في فولدر الداتا بروسيسد

In [10]:
# task 4
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import expr
from pyspark.sql.functions import to_date, col ,date_format , to_timestamp , lower

spark = (
    SparkSession.builder
    .appName("transform3")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)
#==============================================================
# هنجيب الداتا  ونحولها لداتا فريم

df = spark.read.csv(
    "s3a://datalake/raw/Centerline.csv",
    header=True,
    inferSchema=True
)

#--------------------------------------
#هنجيب الاعمده الي محتجينها بس 
df=df.select("STREET NAME","GlobalID","Number Park Lanes","Number Travel Lanes","Snow Priority",
          "Street Width","BIKE_LANE")

#--------------------------------------
# fillna
df=df.fillna(
    {
        "Number Park Lanes":0, # تعويض الارقام الي ب نل ب 0
        "Number Travel Lanes":2,
        "Snow Priority":"UNKNOWN",
        "Street Width":30,
        "BIKE_LANE":0

    }
)
#--------------------------------------
# تغير اسماء الاعمده 
rename_dic={
    "STREET NAME":"street_name",
    "GlobalID":"global_id",
    "Number Park Lanes":"number_park_lanes",
    "Number Travel Lanes":"number_travel_lanes",
    "Snow Priority":"snow_priority",
    "Street Width":"street_width",
    "BIKE_LANE":"bike_lane"

}

for old , new in rename_dic.items():
     df=df.withColumnRenamed(old, new)
#  هنحول القيم الي في الاعمده الاسترينج الي قيم كلها سمول 
df = df.withColumn("street_name",lower(col("street_name")))


df.show(10)
#--------------------------------------
# هنحفظ الفايل الجديد علي س3 في الفولدر بتاع الداتا المتعالجه 
df.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv("s3a://datalake/processed/streets_data")

+------------------+--------------------+-----------------+-------------------+-------------+------------+---------+
|       street_name|           global_id|number_park_lanes|number_travel_lanes|snow_priority|street_width|bike_lane|
+------------------+--------------------+-----------------+-------------------+-------------+------------+---------+
|                 n|cedc2dde-7e8b-442...|                2|                  2|            C|          34|        0|
|              hone|9c163e85-23ad-418...|                2|                  2|            S|          32|        0|
|                48|fdccf94f-201f-431...|                2|                  1|            S|          32|        0|
|            laight|bcbbb800-b963-45b...|                2|                  1|            S|          34|        0|
|             brook|d7f8c5d8-637b-412...|                1|                  2|            H|          25|        0|
|                60|27fcb089-c93b-41f...|                2|     

26/01/10 17:16:12 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:16:12 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 17:16:12 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


#Task 5
 هنجيب الداتا كلها ونبدء نعمل جوينز عشان نكون الداتا وير هاوس 
 هنرفع الفاكت والديمنشنز علي s3  
 في فولدر جديد اسمه dw 
 

In [21]:
spark.stop()

ConnectionRefusedError: [Errno 111] Connection refused

In [20]:
#task 5
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("dw")

    # ✅ Hadoop S3A jars 
    .config(
        "spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262"
    )

    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9010")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    .getOrCreate()
)

# هنجيب اول داتا ونحطها في داتا فريم streets_data
streets_data = spark.read.csv(
    "s3a://datalake/processed/streets_data",
    header=True,
    inferSchema=True
)

# هنجيب ثاني  داتا ونحطها في داتا فريم Motor_Vehicle_Collisions_Crashes
Motor_Vehicle = spark.read.csv(
    "s3a://datalake/processed/Motor_Vehicle_Collisions_Crashes",
    header=True,
    inferSchema=True
)

# هنجيب ثالث  داتا ونحطها في داتا فريم mongo_traffic
complaint_311 = spark.read.csv(
    "s3a://datalake/processed/complaint_311",
    header=True,
    inferSchema=True
)


#print(streets_data.printSchema())
#print(Motor_Vehicle.printSchema())




ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
# create dim_borough

from pyspark.sql.functions import monotonically_increasing_id
dim_borough=Motor_Vehicle.select("borough").distinct()

#  نضيف عمود ID فريد لكل بورغو
dim_borough = dim_borough.withColumn("borough_id", monotonically_increasing_id())

# 4- نرتب الأعمدة بحيث يكون ID الأول
dim_borough = dim_borough.select("borough_id", "borough")

# 5- نعرض عينة للتأكد
dim_borough.show(10, truncate=False)

dim_borough.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv("s3a://datalake/data_warehouse/dim_borough")


+----------+-------------+
|borough_id|borough      |
+----------+-------------+
|0         |bronx        |
|1         |unknown      |
|2         |manhattan    |
|3         |brooklyn     |
|4         |queens       |
|5         |staten island|
+----------+-------------+



26/01/10 18:49:15 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 18:49:15 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [34]:
# create vecicle_type 

from pyspark.sql.functions import monotonically_increasing_id
dim_vehicle_type=Motor_Vehicle.select("vehicle_type").distinct()

#  نضيف عمود ID فريد لكل سياره 
dim_vehicle_type = dim_vehicle_type.withColumn("vehicle_id", monotonically_increasing_id())

#  نرتب الأعمدة بحيث يكون ID الأول
dim_vehicle_type = dim_vehicle_type.select("vehicle_id", "vehicle_type")

#  نعرض عينة للتأكد
print(dim_vehicle_type.count())

dim_vehicle_type.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv("s3a://datalake/data_warehouse/dim_vehicle_type")


260


26/01/10 20:31:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 20:31:00 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [ ]:
# create weather_dim 

from pyspark.sql.functions import monotonically_increasing_id
dim_weather=complaint_311.select("weather_condition").distinct()

#  نضيف عمود ID فريد لكل طقس 
dim_weather = dim_weather.withColumn("weather_id", monotonically_increasing_id())

#  نرتب الأعمدة بحيث يكون ID الأول
dim_weather = dim_weather.select("weather_id", "weather_condition")

#  نعرض عينة للتأكد
print(dim_weather.count())
dim_weather.show()

dim_weather.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv("s3a://datalake/data_warehouse/dim_weather")


4
+----------+-----------------+
|weather_id|weather_condition|
+----------+-----------------+
|         0|              fog|
|         1|             rain|
|         2|            clear|
|         3|             snow|
+----------+-----------------+



26/01/10 20:38:23 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 20:38:23 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [36]:
# create dim_contributing

dim_contributing=Motor_Vehicle.select("contributing_factor").distinct()

#  نضيف عمود ID فريد لكل سبب حادث 
dim_contributing = dim_contributing.withColumn("contributing_id", monotonically_increasing_id())

#  نرتب الأعمدة بحيث يكون ID الأول
dim_contributing = dim_contributing.select("contributing_id", "contributing_factor")

#  نعرض عينة للتأكد
print(dim_contributing.count())
dim_contributing.show()

dim_contributing.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv("s3a://datalake/data_warehouse/dim_contributing")


56


+---------------+--------------------+
|contributing_id| contributing_factor|
+---------------+--------------------+
|              0|aggressive drivin...|
|              1|windshield inadeq...|
|              2|cell phone (hands...|
|              3|headlights defective|
|              4|   oversized vehicle|
|              5|pedestrian/bicycl...|
|              6|     fatigued/drowsy|
|              7|  pavement defective|
|              8|     other vehicular|
|              9|unsafe lane changing|
|             10|               glare|
|             11|lane marking impr...|
|             12|driver inattentio...|
|             13|following too clo...|
|             14|     drugs (illegal)|
|             15|      tinted windows|
|             16|passenger distrac...|
|             17| tow hitch defective|
|             18|    steering failure|
|             19|  turning improperly|
+---------------+--------------------+
only showing top 20 rows


26/01/10 20:45:34 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/10 20:45:34 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [4]:
# create dim_street 

from pyspark.sql.functions import monotonically_increasing_id
dim_street=streets_data.select(
    "street_name",
    "number_park_lanes",
    "number_travel_lanes",
    "snow_priority",
    "street_width",
    "bike_lane")
#هنمسح  الشوارع المكرره 
dim_street =dim_street.dropDuplicates(['street_name'])

#  نضيف عمود ID فريد لكل شارع  
dim_street = dim_street.withColumn("street_id", monotonically_increasing_id())

#  نرتب الأعمدة بحيث يكون ID الأول
dim_street = dim_street.select(
    "street_id", 
    "street_name",
    "number_park_lanes",
    "number_travel_lanes",
    "snow_priority",
    "street_width",
    "bike_lane")

#  نعرض عينة للتأكد
print(dim_street.count())
dim_street.show(10)

dim_street.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv("s3a://datalake/data_warehouse/dim_street")


7015


+---------+--------------------+-----------------+-------------------+-------------+------------+---------+
|street_id|         street_name|number_park_lanes|number_travel_lanes|snow_priority|street_width|bike_lane|
+---------+--------------------+-----------------+-------------------+-------------+------------+---------+
|        0|                   1|                2|                  1|            H|          28|        0|
|        1|          1 division|                2|                  1|            V|          30|        0|
|        2|                  10|                2|                  1|            S|          28|        0|
|        3|   10 ave pedestrian|                0|                  2|      UNKNOWN|          30|        1|
|        4|10 street pedestrian|                0|                  2|      UNKNOWN|          30|        4|
|        5|                 100|                2|                  2|            C|          65|        0|
|        6|                 

26/01/11 01:25:46 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/11 01:25:46 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [3]:
# create dim_complaint_type

from pyspark.sql.functions import monotonically_increasing_id
dim_complaint_type=complaint_311.select("complaint_type")
    
  
#هنمسح  انواع الشكاوي  المكرره  المكرره 
dim_complaint_type =dim_complaint_type.dropDuplicates(['complaint_type'])

#  نضيف عمود ID فريد لكل شارع  
dim_complaint_type = dim_complaint_type.withColumn("complaint_id", monotonically_increasing_id())

#  نرتب الأعمدة بحيث يكون ID الأول
dim_complaint_type = dim_complaint_type.select(
    "complaint_id", 
    "complaint_type")

#  نعرض عينة للتأكد
print(dim_complaint_type.count())
dim_complaint_type.show(10)

dim_complaint_type.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv("s3a://datalake/data_warehouse/dim_complaint_type")


6


+------------+-------------------+
|complaint_id|     complaint_type|
+------------+-------------------+
|           0|pedestrian accident|
|           1|   vehicle accident|
|           2|        road damage|
|           3| signal malfunction|
|           4|    illegal parking|
|           5| traffic congestion|
+------------+-------------------+



26/01/12 00:13:36 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/12 00:13:37 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [ ]:
# dim_date 
from pyspark.sql import functions as F
from pyspark.sql.window import Window

dim_date = spark.sql("""
    SELECT explode(
        sequence(
            to_date('2025-01-01'),
            to_date('2025-12-31'),
            interval 1 day
        )
    ) AS date
""")
window_spec = Window.orderBy("date")

dim_date = dim_date.select(
    F.row_number().over(window_spec).alias("date_id"),   # key فريد 1 → 365
    F.date_format("date", "dd-MM-yyyy").alias("full_date"),
    F.date_format("date", "EEEE").alias("day_name"),
    F.date_format("date", "MMMM").alias("month_name")
)

# تغير الداتا تايب بتاع الديت 
dim_date = dim_date.withColumn(
    "full_date",
    F.to_date(F.col("full_date"),"dd-MM-yyyy")
)

dim_date.show(10, truncate=False)

dim_date.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv("s3a://datalake/data_warehouse/dim_date")

26/01/12 01:00:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/12 01:00:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/12 01:00:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+----------+---------+----------+
|date_id|full_date |day_name |month_name|
+-------+----------+---------+----------+
|1      |2025-01-01|Wednesday|January   |
|2      |2025-01-02|Thursday |January   |
|3      |2025-01-03|Friday   |January   |
|4      |2025-01-04|Saturday |January   |
|5      |2025-01-05|Sunday   |January   |
|6      |2025-01-06|Monday   |January   |
|7      |2025-01-07|Tuesday  |January   |
|8      |2025-01-08|Wednesday|January   |
|9      |2025-01-09|Thursday |January   |
|10     |2025-01-10|Friday   |January   |
+-------+----------+---------+----------+
only showing top 10 rows


26/01/12 01:00:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/12 01:00:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/12 01:00:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/12 01:00:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/12 01:00:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/12 01:00:20 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
26/01/12 01:00:21 WARN AbstractS3AC

26/01/12 01:18:19 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from 10.0.2.15:43599 in 10000 milliseconds
26/01/12 01:18:19 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Future timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:35)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala

In [ ]:
# create fact_complaint

from pyspark.sql.functions import monotonically_increasing_id
dim_complaint_type=complaint_311.select("complaint_type")
    
  
#هنمسح  انواع الشكاوي  المكرره  المكرره 
dim_complaint_type =dim_complaint_type.dropDuplicates(['complaint_type'])

#  نضيف عمود ID فريد لكل شارع  
dim_complaint_type = dim_complaint_type.withColumn("complaint_id", monotonically_increasing_id())

#  نرتب الأعمدة بحيث يكون ID الأول
dim_complaint_type = dim_complaint_type.select(
    "complaint_id", 
    "complaint_type")

#  نعرض عينة للتأكد
print(dim_complaint_type.count())
dim_complaint_type.show(10)

dim_complaint_type.write \
     .mode("overwrite") \
     .option("header","true") \
     .csv("s3a://datalake/data_warehouse/dim_complaint_type")


In [2]:
print(complaint_311.printSchema())

complaint_311.show(5)

root
 |-- borough: string (nullable = true)
 |-- created_date: date (nullable = true)
 |-- pepole_killed: integer (nullable = true)
 |-- complaint_id: string (nullable = true)
 |-- complaint_type: string (nullable = true)
 |-- pepole_injuries: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- response_time_minutes: integer (nullable = true)
 |-- severity: string (nullable = true)
 |-- status: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- traffic_volume_level: string (nullable = true)
 |-- vehicles_involved: integer (nullable = true)
 |-- weather_condition: string (nullable = true)

None
+---------+------------+-------------+--------------------+------------------+---------------+---------+----------+---------------------+--------+------+---------------+--------------------+-----------------+-----------------+
|  borough|created_date|pepole_killed|        complaint_id|    complaint_type|pepole_injur

In [19]:
print(Motor_Vehicle.printSchema())
#print(streets_data.printSchema())



ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
from pyspark.sql.functions import col

complaint_311.groupBy("borough") \
  .count() \
  .orderBy("count", ascending=False) \
  .show(20,truncate=True)


Motor_Vehicle.groupBy("vehicle_type") \
  .count() \
  .orderBy("count", ascending=False) \
  .show(20,truncate=True)
#df_fill.filter(col("CRASH TIME").isNull()).count()

+-------------+-----+
|      borough|count|
+-------------+-----+
|        bronx| 8377|
|     brooklyn| 8292|
|       queens| 8273|
|    manhattan| 8245|
|staten island| 8175|
+-------------+-----+



+--------------------+-----+
|        vehicle_type|count|
+--------------------+-----+
|               sedan|38346|
|station wagon/spo...|27414|
|                taxi| 2484|
|       pick-up truck| 2013|
|                bike| 1780|
|                 bus| 1599|
|           box truck| 1256|
|          motorcycle|  959|
|               moped|  771|
|tractor truck diesel|  544|
|              e-bike|  542|
|                 van|  521|
|           ambulance|  478|
|          standing s|  342|
|         convertible|  258|
|                dump|  255|
|                  pk|  211|
|   garbage or refuse|  203|
|           e-scooter|  175|
|            flat bed|  127|
+--------------------+-----+
only showing top 20 rows
